In [2]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', None)

In [27]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")
coaching_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")

In [30]:
dailyactivity.rename(columns = {'ActivityDate':'date'}, inplace=True)
wear_time.rename(columns = {'Day':'date'}, inplace=True)
dailyFitbitActiveZoneMinutes.rename(columns = {'Date':'date'}, inplace=True)
dailyHRV.rename(columns = {'SleepDay':'date'}, inplace=True)
fitbitBreathingRate.rename(columns = {'SleepDay':'date'}, inplace = True)
fitbitSkinTemperature.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepDay.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepStageLogInfo.rename(columns = {'SleepDay':'date'}, inplace=True)
heartrate_15min.rename(columns = {'Time':'date'}, inplace=True)

#transfer date
total_data = [wear_time, dailyactivity, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min]
for d in total_data:
    d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
    d['date'] = d['date'].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer heart_rate_15 min
heartrate_15min = heartrate_15min.groupby(by = ['Id','date']).mean().reset_index()

C:\Users\karri\AppData\Local\Temp\ipykernel_37080\2171673065.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')


In [33]:
#delete the date if total minutes wear time smaller than 10
wear_time_larger_10_hours = wear_time[wear_time['TotalMinutesWearTime'] >= 10*60] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
len(wear_time_larger_10_hours['Id'].unique()) #99, the number of total participants

99

In [52]:
merged1 = pd.merge(wear_time_larger_10_hours, dailyactivity, on=['Id','date'], how='left')  # inner join only keeps common dates
merged2 = pd.merge(merged1, sleepDay, on=['Id','date'], how='left')
merged3 = pd.merge(merged2, sleepStageLogInfo, on=['Id','date'], how='left')
merged4 = pd.merge(merged3, heartrate_15min, on=['Id','date'], how='left')
merged5 = pd.merge(merged4, dailyFitbitActiveZoneMinutes, on=['Id','date'], how='left')
merged6 = pd.merge(merged5, dailyHRV, on=['Id','date'], how='left')
merged7 = pd.merge(merged6, fitbitBreathingRate, on=['Id','date'], how='left')
merged_final = pd.merge(merged7, fitbitSkinTemperature, on=['Id','date'], how='left')

In [54]:
#transfer Id to number only
merged_final['Id'] = [items[-3:] for items in merged_final['Id']]
merged_final['Id'] = [int(digit) for digit in merged_final['Id']]

Survey data

In [56]:
#survey data preprocessing
survey_data['health_coach_survey_complete'].unique()
survey_data_clean = survey_data[survey_data['health_coach_survey_complete'] == 2]#select the whose health coaching session is completed

time_stamp = ['consent_and_screening_survey_timestamp','baseline_wellness_behavior_survey_timestamp','postsession_survey_timestamp',
              'health_coach_survey_timestamp','weeks_followup_survey_timestamp','weeks_followup_survey_96ac_timestamp']

for time in time_stamp:
    survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
    survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer record ID
survey_data_clean.rename(columns = {'record_id':'Id'}, inplace=True)
survey_data_clean['Id'] = [int(digit) for digit in survey_data_clean['Id']]

#drop non-use column name

survey_data_clean.drop(columns = 'redcap_survey_identifier', inplace = True)

C:\Users\karri\AppData\Local\Temp\ipykernel_37080\4185533662.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
C:\Users\karri\AppData\Local\Temp\ipykernel_37080\4185533662.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')
C:\Users\karri\AppData\Local\Temp\ipykernel_37080\4185533662.py:9: SettingWithCopyWarning: 
A v

In [57]:
#coaching time processing
#data processing
#coaching_time.drop(columns = 'Participants Name', inplace = True)
coaching_time.rename(columns = {'Health Coach Session':'Appointlet_time', "ID":'Id'}, inplace=True)
coaching_time['Appointlet_time'] = pd.to_datetime(coaching_time['Appointlet_time'], errors = 'coerce' )
coaching_time['Appointlet_time'] = coaching_time['Appointlet_time'].dt.date

In [58]:
#merge it with survey data
survey_data_coaching = pd.merge(survey_data_clean, coaching_time, on=['Id'], how='left')

In [59]:
#check the column
time_stamp = ['Id','Appointlet_time', 'health_coach_survey_timestamp', 'postsession_survey_timestamp','weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp']
coaching_time_compare = survey_data_coaching[time_stamp]
coaching_time_compare

,Id,Appointlet_time,health_coach_survey_timestamp,postsession_survey_timestamp,weeks_followup_survey_timestamp,weeks_followup_survey_96ac_timestamp
0,5,2023-11-22,NaN,2023-11-24,2023-12-06,2024-01-07
1,7,2023-11-30,NaN,2023-12-03,2023-12-16,2024-01-17
2,8,2023-10-19,NaN,2023-10-20,2023-11-03,2023-12-04
3,13,2023-10-24,NaN,2023-10-31,2023-11-20,2024-04-10
4,14,2023-10-31,NaN,2023-11-04,2023-11-19,2023-12-26
...,...,...,...,...,...,...
95,156,2024-04-18,NaT,2024-04-25,2024-05-07,2024-06-07
96,162,2024-05-13,2024-05-27,2024-05-22,2024-06-10,2024-07-11
97,164,2024-05-12,2024-05-13,2024-05-13,2024-05-27,2024-06-24
98,165,2024-06-27,NaT,2024-06-27,2024-07-11,2024-08-08


Merge Fitabase data to survey data

In [60]:
#add another column Fitbit data

merged_final.insert(loc = 2, column = 'survey_date', value = np.nan)
fitbit_data = merged_final.copy()

In [84]:
def assign_survey_date(row, time_lst):
    if pd.notna(time_lst[0]) and pd.notna(time_lst[1]) and time_lst[0] <= row['date'] <= time_lst[1]:
        return 'baseline'
    elif pd.notna(time_lst[1]) and pd.notna(time_lst[2]) and time_lst[1] < row['date'] <= time_lst[2]:
        return 'first_follow_up'
    elif pd.notna(time_lst[2]) and pd.notna(time_lst[3]) and time_lst[2] < row['date'] <= time_lst[3]:
        return 'second_follow_up'
    elif  pd.notna(time_lst[3]) and pd.notna(time_lst[1]) and row['date'] >time_lst[3]:
        return 'after_second_follow_up'
    else:
        return None  # In case no condition matches

In [74]:
def time_lst(individual_survey_data, ind):
    
    baseline = individual_survey_data['baseline_wellness_behavior_survey_timestamp'][ind]
    appoinlet = individual_survey_data['Appointlet_time'][ind]
    first_follow_up = individual_survey_data['weeks_followup_survey_timestamp'][ind]
    second_follow_up = individual_survey_data['weeks_followup_survey_96ac_timestamp'][ind]

    individual_time_lst = [baseline, appoinlet, first_follow_up, second_follow_up]
    
    return individual_time_lst

In [105]:
#for id in
fitbit_id = fitbit_data['Id'].unique()
survey_id = survey_data_coaching['Id'].unique()

for i in range(len(fitbit_id)):

    individual_id = fitbit_id[i]
    if individual_id in survey_id:
        individual_survey_data = survey_data_coaching[survey_data_coaching['Id']==individual_id]
        ind = individual_survey_data.index[0]
        individual_time_lst = time_lst(individual_survey_data,ind)

        individual_fitbit_data = fitbit_data[fitbit_data['Id']==individual_id]
        individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst=individual_time_lst)
        fitbit_data.loc[fitbit_data['Id'] == individual_id, 'survey_date'] = individual_fitbit_data['survey_date'] #replace with new define

C:\Users\karri\AppData\Local\Temp\ipykernel_37080\1209124181.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst=individual_time_lst)


In [110]:
fitbit_data

,Id,date,survey_date,TotalMinutesWearTime,PercentageWearTime,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,...,CardioActiveZoneMinutes,PeakActiveZoneMinutes,DailyRMSSD,DeepRMSSD,AvgBreathsPerMinute,LightSleepAvgBreathsPerMinute,DeepSleepAvgBreathsPerMinute,REMSleepAvgBreathsPerMinute,NightlyRelative,LogType
0,5,2023-11-15,baseline,1440,100.000000,18236,13.93,13.93,4.96885,7.96,...,12.0,58.0,35.062,34.852,16.4,16.6,16.4,17.0,NaN,NaN
1,5,2023-11-16,baseline,1214,84.305556,14563,11.43,11.43,5.10630,6.00,...,14.0,56.0,27.656,34.609,17.0,16.8,17.0,16.4,NaN,NaN
2,5,2023-11-17,baseline,1400,97.222222,14770,10.35,10.35,0.00000,3.14,...,6.0,0.0,20.702,25.809,18.0,18.0,18.0,18.2,0.0,other_sensors
3,5,2023-11-18,baseline,876,60.833333,6609,5.72,5.72,4.90004,4.81,...,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2023-11-19,baseline,1429,99.236111,2867,1.94,1.94,0.00000,0.00,...,6.0,0.0,28.155,29.009,18.6,17.8,18.6,-1.0,1.3,other_sensors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,167,2024-07-23,second_follow_up,1398,97.083333,10230,7.08,7.08,0.00000,1.44,...,4.0,0.0,55.328,55.364,18.8,18.0,18.8,18.0,0.3,other_sensors
6476,167,2024-07-24,second_follow_up,1440,100.000000,5131,3.54,3.54,0.00000,0.00,...,2.0,0.0,52.261,61.657,18.6,17.8,18.6,-1.0,0.0,other_sensors
6477,167,2024-07-25,second_follow_up,720,50.000000,3197,2.21,2.21,0.00000,0.99,...,4.0,0.0,53.556,58.256,18.4,18.4,-1.0,20.4,-0.4,other_sensors
6478,167,2024-07-30,second_follow_up,1421,98.680556,13574,9.18,9.18,0.00000,3.34,...,6.0,0.0,53.009,56.143,20.0,17.8,20.0,17.6,0.2,other_sensors
